In [41]:
from bs4 import BeautifulSoup
import pandas as pd
import pathlib, lxml, html5lib
 

First, let's create our dataset. To do so, we loop through all the files in `source_files` and build a soup for each file. At the end of the loop, we should have a single dataset.

In [42]:
def readHTML(path):
    with open(path, encoding="utf-8") as f:
        soup = BeautifulSoup(f, "lxml")
        nodes = soup.select(".node:not(.node-vorschlag-diverse)")
    return nodes


In [43]:
def createSubDataset(nodes, path):
    buergerhaushaelter = []

    for n in nodes:
        nodeNr = n.select_one(".node-nr ").get_text().replace("Nr.", "").strip() if n.select_one(".node-nr ") != None else None

        if nodeNr == None:
           continue

        title  = n.select_one("header .node__title.node-title a").get_text() if n.select_one("header .node__title.node-title a") != None else None
        author = n.select_one("header .submitted .username").get_text() if n.select_one("header .submitted .username") != None else None
        year = int(n.select_one(".field-tax-jahr.inline .field-items .field-item.even").get_text()) if n.select_one(".field-tax-jahr.inline .field-items .field-item.even") != None else None
        stadtbezirk = n.select_one("header .meta .terms .field-tax-stadtteil .field-items .field-item.even").get_text() if n.select_one("header .meta .terms .field-tax-stadtteil .field-items .field-item.even") != None else None
        thema = n.select_one("header .meta .terms .field-tax-thema .field-items .field-item.even").get_text() if n.select_one("header .meta .terms .field-tax-thema .field-items .field-item.even") != None else None
        wirkung = n.select_one("header .meta .terms .field-tax-wirkung .field-items .field-item.even").get_text() if n.select_one("header .meta .terms .field-tax-wirkung .field-items") != None else None
        rank = int(n.select_one(".field-platz.inline .field-items .field-item.even").get_text()) if n.select_one(".field-platz.inline .field-items .field-item.even") != None else None
        status = n.select_one(".group-umsetzung .field-taxo-umsetzung .field-items .field-item.even .taxonomy-term.vocabulary-status").get_text() if n.select_one(".group-umsetzung .field-taxo-umsetzung .field-items .field-item.even .taxonomy-term.vocabulary-status") != None else None
        score_wenigergut = int(n.select_one(".form-item.form-type-item .stuttgart-score .final-stuttgart-score-negative .stuttgart-negative-score").get_text()) if n.select_one(".form-item.form-type-item .stuttgart-score .final-stuttgart-score-negative .stuttgart-negative-score") != None else None
        score_gut = int(n.select_one(".form-item.form-type-item .stuttgart-score .final-stuttgart-score-positive .stuttgart-positive-score").get_text()) if n.select_one(".form-item.form-type-item .stuttgart-score .final-stuttgart-score-positive .stuttgart-positive-score") != None else None
        ergebnis_haushaltsberatungen = n.select_one(".group-umsetzung .field-verwaltungshinweis .field-ergebnis-rat .field-items .field-item.even").get_text() if n.select_one(".group-umsetzung .field-verwaltungshinweis .field-ergebnis-rat .field-items .field-item.even") != None else None
        stellungnahme_verwaltung = n.select_one(".group-umsetzung .field-verwaltungshinweis .field-body .field-items .field-item.even").get_text() if n.select_one(".group-umsetzung .field-verwaltungshinweis .field-body .field-items .field-item.even") != None else None
        bezirksbeiratshinweis = n.select_one(".group-umsetzung .field-bezirksbeiratshinweis .field-items .field-item.even").get_text() if n.select_one(".group-umsetzung .field-bezirksbeiratshinweis .field-items .field-item.even") != None else None
        gemeinderat_prueft = n.select_one(".field-rat-prueft.inline .field-items .field-item.even").get_text() if n.select_one(".field-rat-prueft.inline .field-items .field-item.even") != None else None

        if title != None or author != None or year != None or stadtbezirk != None or thema != None or wirkung != None or rank != None or status != None or score_wenigergut != None or score_gut != None or ergebnis_haushaltsberatungen != None or stellungnahme_verwaltung != None or bezirksbeiratshinweis != None or gemeinderat_prueft != None:
          buergerhaushaelter.append([title, nodeNr, author,  stadtbezirk, year, thema, wirkung, rank, status, score_wenigergut, score_gut, ergebnis_haushaltsberatungen, stellungnahme_verwaltung, bezirksbeiratshinweis, gemeinderat_prueft])


    buergerhaushaelter_df = pd.DataFrame(buergerhaushaelter, columns=['title', 'node nr.', 'author', 'stadtbezirk', 'year', 'thema', 'wirkung', 'rank', 'status', 'score_wenigergut', 'score_gut', 'ergebnis_haushaltsberatungen', 'stellungnahme_verwaltung', 'bezirksbeiratshinweis', 'gemeinderat_prueft'])

    buergerhaushaelter_df['path'] = path

    #print(buergerhaushaelter_df)
    return buergerhaushaelter_df

In [ ]:
file_paths = [str(path) for path in list((pathlib.Path("./source_files/")).iterdir())]

dataset = pd.DataFrame()

for path in file_paths:
    print(path)
    nodes = readHTML(path)
    dataset = pd.concat([dataset, createSubDataset(nodes, path)])


source_files/buergerhaushalt_27.html
source_files/buergerhaushalt_31.html


In [ ]:
dataset.drop_duplicates(inplace = True)
dataset.reset_index(inplace=True)
dataset.drop(columns = "index", inplace = True)

In [ ]:
print(dataset)

                                                   title node nr.  \
0      S21 / Risiken / Sicherer Schutz vor Feuer und ...     4639   
1         Angebote für Jugendliche in Vaihingen schaffen     4350   
2                                  SSB Anzeigen abändern     3859   
3             Innenstadtzone bis ZOB Degerloch ausweiten     5162   
4             Ein Mobilfunkvorsorgekonzept für Stuttgart     3904   
...                                                  ...      ...   
15506  Bepflanzung Verkehrsinsel  am Kreisverkehr Mah...    60403   
15507     Müllsäuberungsaktion im Tapachtal organisieren    60830   
15508               Unterstützung Projekt Boulderblöckle    62283   
15509            Verkehrkonzept für Feuerbach verbessern    61683   
15510                      breitere Gehwege installieren    61264   

                   author                         stadtbezirk    year  \
0                    Gast  \n        Stuttgart (gesamt)        2013.0   
1                 Pinguin